In [ ]:
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import metrics
from utils import *
from model import *

data1 = pd.read_csv("./WFC.csv")
data1.index = pd.to_datetime(data1['Date'])
#data1 = data1.drop(['ts_code', 'trade_date', 'turnover_rate', 'volume_ratio', 'pb', 'total_share', 'float_share', 'free_share'], axis=1)
data1 = data1.loc[:, ['Open', 'High', 'Low', 'Close', 'Adj Close','Volume']]
data_yuan = data1
residuals = pd.read_csv('./ARIMA_residuals1.csv')
residuals.index = pd.to_datetime(residuals['Date'])
residuals.pop('Date')
data1 = pd.merge(data1, residuals, on='Date')
data = data1.iloc[1:3658, :] 
data2 = data1.iloc[3658:, :] 

TIME_STEPS = 20

data, normalize = NormalizeMult(data)
print('#', normalize)
pollution_data = data[:, 3].reshape(len(data), 1)

train_X, _ = create_dataset(data, TIME_STEPS)
_, train_Y = create_dataset(pollution_data, TIME_STEPS)

print(train_X.shape, train_Y.shape)

m = attention_model(INPUT_DIMS=7)
m.summary() 
adam = Adam(learning_rate=0.01)
m.compile(optimizer=adam, loss='mse') 
history = m.fit([train_X], train_Y, epochs=50, batch_size=32, validation_split=0.1)
m.save("./stock_model.h5")
np.save("stock_normalize.npy", normalize)
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# normalize = np.load("normalize.npy")
# loadmodelname = "model.h5"

class Config:
    def __init__(self):
        self.dimname = 'Close'

config = Config()
name = config.dimname
# normalize = np.load("normalize.npy")
y_hat, y_test = PredictWithData(data2, data_yuan, name, 'stock_model.h5',7)
y_hat = np.array(y_hat, dtype='float64')
y_test = np.array(y_test, dtype='float64')
evaluation_metric(y_test,y_hat)

2023-10-23 13:00:41.462291: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-23 13:00:41.463979: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 13:00:41.494688: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 13:00:41.496831: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 13:00:42.431705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

(7, 2)
# [[ 8.65000000e+00  6.58899990e+01]
 [ 8.94000000e+00  6.63099980e+01]
 [ 7.80000000e+00  6.56600040e+01]
 [ 8.12000000e+00  6.59300000e+01]
 [ 5.63584400e+00  5.58309710e+01]
 [ 3.65440000e+06  4.78736600e+08]
 [-6.05427759e+00  6.61294887e+00]]
(3636, 20, 7) (3636, 1)


2023-10-23 13:00:44.744373: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-10-23 13:00:44.937553: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-23 13:00:44.938393: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMEN

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 7)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 20, 64)       512         ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 20, 64)       0           ['conv1d[0][0]']                 
                                                                                                  
 bidirectional (Bidirectional)  (None, 20, 128)      66048       ['dropout[0][0]']                
                                                                                              

2023-10-23 13:00:45.029697: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-10-23 13:00:45.061987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-23 13:00:45.062794: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

 98/103 [===========================>..] - ETA: 0s - loss: 0.0042

2023-10-23 13:00:48.075007: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-23 13:00:48.076072: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-23 13:00:48.076846: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

103/103 [==============================] - 3s 13ms/step - loss: 0.0041 - val_loss: 0.0033
Epoch 2/50
103/103 [==============================] - 1s 8ms/step - loss: 0.0013 - val_loss: 0.0035
Epoch 3/50
103/103 [==============================] - 1s 8ms/step - loss: 0.0011 - val_loss: 0.0028
Epoch 4/50
103/103 [==============================] - 1s 8ms/step - loss: 0.0010 - val_loss: 0.0018
Epoch 5/50
103/103 [==============================] - 1s 8ms/step - loss: 7.8990e-04 - val_loss: 0.0017
Epoch 6/50
103/103 [==============================] - 1s 8ms/step - loss: 8.1566e-04 - val_loss: 0.0022
Epoch 7/50
103/103 [==============================] - 1s 8ms/step - loss: 7.1282e-04 - val_loss: 0.0016
Epoch 8/50
103/103 [==============================] - 1s 8ms/step - loss: 7.6394e-04 - val_loss: 8.1007e-04
Epoch 9/50
103/103 [==============================] - 1s 8ms/step - loss: 6.7136e-04 - val_loss: 0.0021
Epoch 10/50
103/103 [==============================] - 1s 8ms/step - loss: 6.7323e-04 

In [ ]:
time = pd.Series(data1.index[3659:])
plt.figure(figsize=(10, 6))
plt.plot(time, y_test, label='True')
plt.plot(time, y_hat, label='Prediction')
plt.title('Hybrid model prediction')
plt.xlabel('Time', fontsize=12, verticalalignment='top')
plt.ylabel('Price', fontsize=14, horizontalalignment='center')
plt.legend()
plt.show()